# 전처리

In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 4.18 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 553 kB/s 
     |████████████████████████████████| 453 kB 67.6 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-06-08 03:18:41--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22e9:9f55, ...
Connecting to bitbucket.org (bi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_excel('/content/한국어_단발성_대화_데이터셋.xlsx', usecols = [0, 1])
data

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [ ]:
data['Sentence'] = data['Sentence'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣 ]", "", regex=True)
data['Sentence'] = data['Sentence'].str.replace('^ +', "", regex=True)
data['Sentence'].replace('', np.nan, inplace=True)
data = data.dropna(how='any').reset_index(drop=True)
data

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요,공포
1,그냥 내 느낌일뿐겠지,공포
2,아직너무초기라서 그런거죠,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38587,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라,혐오
38588,재미가 없으니 망하지,혐오
38589,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38590,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [ ]:
f = open('/content/한국어불용어100.txt', 'r', encoding='utf8')
stopwords = []
for i in range(100):
  line = f.readline()
  line = line.split('\t')
  stopwords.append(line[0])
f.close()

In [ ]:
!pip install conlpy
import re
from konlpy.tag import Okt, Komoran

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement conlpy (from versions: none)
ERROR: No matching distribution found for conlpy


In [ ]:
okt = Okt()

def mtokenize(sentence):
  result = []
  tokens = okt.morphs(sentence)
  for word in tokens:
    if word not in stopwords:
      result.append(word)
  if len(result) != 0:
    return result

In [ ]:
data['tokens'] = data['Sentence'].apply(lambda x: mtokenize(x))
print(data['tokens'])
data = data.dropna().reset_index(drop=True)

0                              [언니, 동생, 으로, 부르는게, 맞는, 인가요]
1                                          [그냥, 느낌, 뿐, 겠지]
2                                   [아직, 너무, 초기, 라서, 거, 죠]
3                                      [유치원, 버스, 사고, 낫다던데]
4                                       [근데, 원래, 이런거, 맞나요]
                               ...                        
38587      [솔직히, 예보, 제대로, 못, 하는, 세금, 이라도, 아끼게, 그냥, 폐지, 해라]
38588                                       [재미, 없으니, 망하지]
38589    [공장, 도시락, 비우, 생, 적임, 아르바이트, 했는데, 화장실, 가성, 손, 도...
38590      [코딱지, 만, 나라, 에서, 끼리, 피터지게, 싸우는, 센, 징, 클래스, ㅉㅉㅉ]
38591      [와이프, 도, 그렇고, 댓글, 다, 볼텐데, 이휘재, 하차, 하라, 고, 해주세요]
Name: tokens, Length: 38592, dtype: object


In [ ]:
data

,Sentence,Emotion,tokens
0,언니 동생으로 부르는게 맞는 일인가요,공포,"[언니, 동생, 으로, 부르는게, 맞는, 인가요]"
1,그냥 내 느낌일뿐겠지,공포,"[그냥, 느낌, 뿐, 겠지]"
2,아직너무초기라서 그런거죠,공포,"[아직, 너무, 초기, 라서, 거, 죠]"
3,유치원버스 사고 낫다던데,공포,"[유치원, 버스, 사고, 낫다던데]"
4,근데 원래이런거맞나요,공포,"[근데, 원래, 이런거, 맞나요]"
...,...,...,...
38583,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라,혐오,"[솔직히, 예보, 제대로, 못, 하는, 세금, 이라도, 아끼게, 그냥, 폐지, 해라]"
38584,재미가 없으니 망하지,혐오,"[재미, 없으니, 망하지]"
38585,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,"[공장, 도시락, 비우, 생, 적임, 아르바이트, 했는데, 화장실, 가성, 손, 도..."
38586,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,"[코딱지, 만, 나라, 에서, 끼리, 피터지게, 싸우는, 센, 징, 클래스, ㅉㅉㅉ]"


In [ ]:
data['label'] = data['Emotion'].replace('공포', 0).replace('놀람', 1).replace('분노', 2).replace('슬픔', 3).replace('중립', 4).replace('행복', 5).replace('혐오', 6)

In [ ]:
data

,Sentence,Emotion,tokens,label
0,언니 동생으로 부르는게 맞는 일인가요,공포,"[언니, 동생, 으로, 부르는게, 맞는, 인가요]",0
1,그냥 내 느낌일뿐겠지,공포,"[그냥, 느낌, 뿐, 겠지]",0
2,아직너무초기라서 그런거죠,공포,"[아직, 너무, 초기, 라서, 거, 죠]",0
3,유치원버스 사고 낫다던데,공포,"[유치원, 버스, 사고, 낫다던데]",0
4,근데 원래이런거맞나요,공포,"[근데, 원래, 이런거, 맞나요]",0
...,...,...,...,...
38583,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라,혐오,"[솔직히, 예보, 제대로, 못, 하는, 세금, 이라도, 아끼게, 그냥, 폐지, 해라]",6
38584,재미가 없으니 망하지,혐오,"[재미, 없으니, 망하지]",6
38585,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,"[공장, 도시락, 비우, 생, 적임, 아르바이트, 했는데, 화장실, 가성, 손, 도...",6
38586,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,"[코딱지, 만, 나라, 에서, 끼리, 피터지게, 싸우는, 센, 징, 클래스, ㅉㅉㅉ]",6


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['tokens'])

In [ ]:
vocab_size = len(tokenizer.word_index)+1

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(data['tokens'])

In [ ]:
x = tokenizer.texts_to_sequences(data['tokens'])
y = data['label']

In [ ]:
max = max([len(i) for i in x])
x = pad_sequences(x, maxlen = max)
max

112

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

In [ ]:
Y = data['Emotion']

X_train, X_test, Y_train, Y_test = train_test_split(x, Y, test_size=0.2, random_state=5)

In [ ]:
print(x_train.shape, y_train.shape)

(30870, 112) (30870,)


# CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='mse', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), callbacks=[es, mc])

Epoch 1/100
965/965 [==============================] - ETA: 0s - loss: 7.9751 - acc: 0.1529
Epoch 1: val_acc improved from -inf to 0.15250, saving model to best_model.h5
965/965 [==============================] - 227s 233ms/step - loss: 7.9751 - acc: 0.1529 - val_loss: 8.0439 - val_acc: 0.1525
Epoch 2/100
965/965 [==============================] - ETA: 0s - loss: 7.9650 - acc: 0.1528
Epoch 2: val_acc did not improve from 0.15250
965/965 [==============================] - 222s 230ms/step - loss: 7.9650 - acc: 0.1528 - val_loss: 8.0439 - val_acc: 0.1525
Epoch 3/100
965/965 [==============================] - ETA: 0s - loss: 7.9650 - acc: 0.1528
Epoch 3: val_acc did not improve from 0.15250
965/965 [==============================] - 221s 229ms/step - loss: 7.9650 - acc: 0.1528 - val_loss: 8.0439 - val_acc: 0.1525
Epoch 4/100
965/965 [==============================] - ETA: 0s - loss: 7.9650 - acc: 0.1528
Epoch 4: val_acc did not improve from 0.15250
965/965 [==============================] 

In [ ]:
model.evaluate(x_test, y_test)

242/242 [==============================] - 7s 28ms/step - loss: 8.0439 - acc: 0.1525


[8.043923377990723, 0.15250064432621002]

# 머신러닝

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=100) #모델 생성
knn.fit(x_train, y_train) #훈련
knn.score(x_test, y_test)

0.3937548587717025

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_ = KNeighborsClassifier(n_neighbors=100) #모델 생성
knn_.fit(X_train, Y_train) #훈련
knn_.score(X_test, Y_test)

0.3937548587717025

In [ ]:
from sklearn.naive_bayes import GaussianNB
gNB = GaussianNB().fit(x_train, y_train)
gNB.score(x_test, y_test)

0.1611816532780513

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=5000).fit(x_train, y_train)
logreg.score(x_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.1681782845296709

In [ ]:
 from sklearn.svm import LinearSVC
 svc= LinearSVC(max_iter=5000).fit(x_train, y_train)
 svc.score(x_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


0.15340761855402954

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)
tree.score(x_test, y_test)

0.6259393625291526

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(random_state=0, learning_rate=0.1)
gbrt.fit(x_train, y_train)
gbrt.score(x_test, y_test)

0.6785436641617

# LSTM

In [ ]:
es2 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc2 = ModelCheckpoint('best_model2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
from tensorflow.keras.layers import LSTM

model2 = Sequential()
model2.add(Embedding(vocab_size, 100))
model2.add(LSTM(100, recurrent_dropout=0.5))
model2.add(Dropout(0.8))
model2.add(Dense(64))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='rmsprop', loss='mse', metrics=['acc'])
history2 = model2.fit(x_train, y_train, epochs=100, callbacks=[es2, mc2], batch_size=60, validation_data=(x_test, y_test))

Epoch 1/100
515/515 [==============================] - ETA: 0s - loss: 7.9760 - acc: 0.1529
Epoch 1: val_acc improved from -inf to 0.15250, saving model to best_model2.h5
515/515 [==============================] - 153s 291ms/step - loss: 7.9760 - acc: 0.1529 - val_loss: 8.0439 - val_acc: 0.1525
Epoch 2/100
515/515 [==============================] - ETA: 0s - loss: 7.9650 - acc: 0.1528
Epoch 2: val_acc did not improve from 0.15250
515/515 [==============================] - 153s 297ms/step - loss: 7.9650 - acc: 0.1528 - val_loss: 8.0439 - val_acc: 0.1525
Epoch 3/100
515/515 [==============================] - ETA: 0s - loss: 7.9650 - acc: 0.1528
Epoch 3: val_acc did not improve from 0.15250
515/515 [==============================] - 150s 292ms/step - loss: 7.9650 - acc: 0.1528 - val_loss: 8.0439 - val_acc: 0.1525
Epoch 4/100
515/515 [==============================] - ETA: 0s - loss: 7.9650 - acc: 0.1528
Epoch 4: val_acc did not improve from 0.15250
515/515 [==============================]

In [ ]:
model2.evaluate(x_test, y_test)

242/242 [==============================] - 8s 32ms/step - loss: 8.0439 - acc: 0.1525


[8.043923377990723, 0.15250064432621002]